In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import ast  # Pour convertir les chaînes d'annotations en liste
from sklearn.model_selection import train_test_split


In [2]:
# Charger le fichier CSV contenant les données
data = pd.read_csv('coarse-and-fine-grained-ner-dataset (1).csv')

# Afficher les premières lignes pour vérifier la structure
print(data.head())



                                                Text  \
0   grandes feuilles opposées, oblongues-elliptiq...   
1   feuilles opposées, groupées à l'extrémité des...   
2   feuilles opposées, obovées oblongues, arrondi...   
3   arbustes  petites feuilles opposées, groupées...   
4   arbustes  feuilles opposées ou alternes, obla...   

                                      Organ Entities  \
0  ['bouton', 'pédicelle', 'corolle', 'tube', 'fe...   
1  ['limbe', 'style', 'filets', 'rameaux', 'sépal...   
2  ['corolle', 'limbe', 'ovaire', 'lobes', 'base'...   
3  ['anthères', 'pétales', 'tube', 'feuilles', 's...   
4  ['base', 'nervure', 'feuilles', 'arbustes', 'l...   

                                 Descriptor Entities  \
0  ['fermée', 'pubes-cents', 'cunéiformes', 'vent...   
1  ['elliptiques', '1 cm de longueur', 'extrorses...   
2  ['cunéiforme', '10,5 mm de longueur', 'long', ...   
3  ['secondaires', 'accusé', 'saillantes', 'apicu...   
4  ['proéminente', 'décurrente', 'alternes', '

In [3]:
import ast

# Conversion des annotations et préparation des données
tagged_sentences = []

# Parcourir les lignes du DataFrame
for index, row in data.iterrows():
    text = row['Text']
    annotations = row['Fine-grained Annotation']

    # Convertir en liste de tuples en toute sécurité
    try:
        annotations = ast.literal_eval(annotations)
    except (ValueError, SyntaxError):
        print(f"Erreur dans l'annotation à l'index {index}")
        continue

    # Extraire les entités
    entities = [(text[start:end], label) for start, end, label in annotations]
    tagged_sentences.append((text, entities))

# Afficher les premières phrases pour vérification
for i in range(min(5, len(tagged_sentences))):
    print(f"Phrase {i+1}: {tagged_sentences[i]}")


Phrase 1: (" grandes feuilles opposées, oblongues-elliptiques ou obovées-elliptiques, arrondies au sommet, obtuses ou cunéiformes à la base  limbe glabre, mesurant jusqu'à 20 cm de longueur sur 12 cm de largeur  nervure médiane proéminente dessous, un peu saillante dessus  nervures secondaires, 5 à 10 paires, incurvées, réunies en arceaux assez loin de la marge, saillantes dessous, bien marquées dessus, anastomosées à un réseau de nervilles à grosses mailles irrégulières, finement saillant dessus  pétiole 5-20 mm  fleurs blanches fasciculées sur le vieux bois  pédicelle 4-6 mm, glabre ou légèrement pubescent  galice : 4 sépales (2 + 2) de 2,5 mm, un peu pubes-cents extérieurement  corolle à 8 lobes de 3 mm; tube 2 mm  ëtamines 8, insérées à la gorge; filets 3 mm  ovaire velu, à 4 loges, prolongé d'un long style glabre  dans le bouton la corolle, étroitement fermée, laisse poindre très apparemment le style  fruits inconnus  le spécimen type renferme une seule graine fusiforme non caréné

In [4]:
# Initialiser X et Y
X, Y = [], []

# Préparer les données avec ajout du tag "OUT" pour les mots non annotés
for sentence in tagged_sentences:
    text, entities = sentence
    if not entities:  # Vérification si `entities` est vide
        continue

    # Initialiser une liste de mots et de tags
    X_sentence = []
    Y_sentence = []

    # Ajouter les entités avec leurs tags
    for word, label in entities:
        X_sentence.append(word.lower())  # Ajouter le mot en minuscules
        Y_sentence.append(label)  # Ajouter le tag correspondant

    # Ajouter les mots non annotés avec le tag "OUT"
    words = text.split()  # Diviser le texte en mots
    for word in words:
        if word.lower() not in [entity[0].lower() for entity in entities]:  # Si le mot n'est pas déjà dans les entités
            X_sentence.append(word.lower())
            Y_sentence.append("OUT")  # Tag "OUT" pour les mots non annotés

    X.append(X_sentence)
    Y.append(Y_sentence)

# Vérifier les résultats
print(f"Exemple de X : {X[:2]}")
print(f"Exemple de Y : {Y[:2]}")


Exemple de X : [['pubes-cents', 'anastomosées', 'sépales', 'bords', 'cicatrice', 'incurvées', 'secondaires', 'obovées-elliptiques', 'blanches', 'glabre', 'glabre', 'style', 'fleurs', 'bouton', 'fruits', '12 cm de largeur', 'fermée', 'feuilles', 'filets', 'pubescent', 'bois', 'marge', 'nervure', 'fasciculées', 'lobes', 'limbe', 'carénée', 'velu', 'crénelés', 'base', 'saillante', 'pédicelle', 'proéminente', 'ovaire', 'cunéiformes', 'corolle', 'graine', 'glabre', '20 cm de longueur', 'oblongues-elliptiques', 'nervilles', 'ventrale', 'tube', 'opposées', 'loges', 'style', 'réseau', 'corolle', 'insérées', 'pétiole', 'obtuses', 'fusiforme', 'laisse', 'grandes', 'opposées,', 'ou', 'obovées-elliptiques,', 'arrondies', 'au', 'sommet,', 'ou', 'à', 'la', 'glabre,', 'mesurant', "jusqu'à", '20', 'cm', 'de', 'longueur', 'sur', '12', 'cm', 'de', 'largeur', 'médiane', 'dessous,', 'un', 'peu', 'dessus', 'nervures', 'secondaires,', '5', 'à', '10', 'paires,', 'incurvées,', 'réunies', 'en', 'arceaux', 'ass

In [5]:
# Créer le vocabulaire et les tags
vocab = {word.lower() for sentence in X for word in sentence}
tags = {tag for sentence in Y for tag in sentence}

# Ajouter le tag "OUT" dans les tags
tags.add("OUT")

print(f"Taille du vocabulaire : {len(vocab)}")
print(f"Nombre total de tags : {len(tags)}")
print(f"Tags disponibles : {tags}")


Taille du vocabulaire : 13496
Nombre total de tags : 11
Tags disponibles : {'DISPOSITION', 'FORME', 'SURFACE', 'OUT', 'COULEUR', 'DESCRIPTEUR', 'MESURE', 'STRUCTURE', 'DEVELOPPEMENT', 'POSITION', 'ORGANE'}


In [6]:
# Créer les dictionnaires
word_to_ix = {word: idx for idx, word in enumerate(vocab, start=0)}
tag_to_ix = {tag: idx for idx, tag in enumerate(tags, start=0)}

# Ajouter le token <PAD>
PAD_TOKEN = "<PAD>"
word_to_ix[PAD_TOKEN] = len(word_to_ix)
tag_to_ix[PAD_TOKEN] = len(tag_to_ix)

# Vérifier les dictionnaires
print(f"Dictionnaire des mots : {list(word_to_ix.items())[:10]}...")  # Afficher les 10 premiers
print(f"Dictionnaire des tags : {tag_to_ix}")


Dictionnaire des mots : [('30 cm de diamètre', 0), ('robuste,', 1), ('rou-', 2), ('6-30-flores,', 3), ('2,5-11,5', 4), ('fonctionnellement', 5), ('1 mm de largeur', 6), ('jonction', 7), ('100-180 mm de long', 8), ("d'éta-mines", 9)]...
Dictionnaire des tags : {'DISPOSITION': 0, 'FORME': 1, 'SURFACE': 2, 'OUT': 3, 'COULEUR': 4, 'DESCRIPTEUR': 5, 'MESURE': 6, 'STRUCTURE': 7, 'DEVELOPPEMENT': 8, 'POSITION': 9, 'ORGANE': 10, '<PAD>': 11}


In [7]:
import torch

# Préparer les séquences pour le modèle
def prepare_sequence(sequence, word_to_ix):
    return torch.tensor(
        [word_to_ix.get(word, word_to_ix[PAD_TOKEN]) for word in sequence],
        dtype=torch.long
    )

def prepare_tag_sequence(tags, tag_to_ix):
    return torch.tensor(
        [tag_to_ix.get(tag, tag_to_ix[PAD_TOKEN]) for tag in tags],
        dtype=torch.long
    )


In [8]:
from sklearn.model_selection import train_test_split

SPLIT_SIZE = 0.2

# Diviser l'ensemble complet de données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=SPLIT_SIZE, random_state=4)

# Diviser les données d'entraînement en ensembles d'entraînement et de validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=SPLIT_SIZE, random_state=4)

# Afficher le nombre d'exemples dans chaque ensemble
print("TRAINING DATA")
print('Number of sequences: {}'.format(len(X_train)))
print("-"*50)
print("TESTING DATA")
print('Number of sequences: {}'.format(len(X_test)))
print("-"*50)
print("VALIDATION DATA")
print('Number of sequences: {}'.format(len(X_val)))


TRAINING DATA
Number of sequences: 535
--------------------------------------------------
TESTING DATA
Number of sequences: 168
--------------------------------------------------
VALIDATION DATA
Number of sequences: 134


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VanillaLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(VanillaLSTMCell, self).__init__()
        self.hidden_size = hidden_size

        # GATES
        self.Wxi = nn.Linear(input_size, hidden_size)
        self.Whi = nn.Linear(hidden_size, hidden_size)

        self.Wxf = nn.Linear(input_size, hidden_size)
        self.Whf = nn.Linear(hidden_size, hidden_size)

        self.Wxo = nn.Linear(input_size, hidden_size)
        self.Who = nn.Linear(hidden_size, hidden_size)

        self.Wxg = nn.Linear(input_size, hidden_size)
        self.Whg = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h_prev, c_prev):
        i_t = torch.sigmoid(self.Wxi(x) + self.Whi(h_prev))
        f_t = torch.sigmoid(self.Wxf(x) + self.Whf(h_prev))
        o_t = torch.sigmoid(self.Wxo(x) + self.Who(h_prev))
        g_t = torch.tanh(self.Wxg(x) + self.Whg(h_prev))

        c_t = f_t * c_prev + i_t * g_t
        h_t = o_t * torch.tanh(c_t)

        return h_t, c_t

class POSTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, dropout=0.5):
        super(POSTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Couche d'embedding
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Vanilla LSTM Cell (cellule personnalisée)
        self.lstm_cell = VanillaLSTMCell(input_size=embedding_dim, hidden_size=hidden_dim)

        # Couche fully connected (FC) pour prédire les étiquettes
        self.fc = nn.Linear(hidden_dim, tagset_size)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, sentence):
        # Obtenir les embeddings
        embeds = self.word_embeddings(sentence)  # Shape: (batch_size, seq_len, embedding_dim)

        batch_size, seq_len, _ = embeds.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embeds.device)
        c_t = torch.zeros(batch_size, self.hidden_dim, device=embeds.device)

        outputs = []
        for t in range(seq_len):
            h_t, c_t = self.lstm_cell(embeds[:, t, :], h_t, c_t)  # Utilisation de la cellule LSTM personnalisée
            outputs.append(h_t)

        # Stack outputs and apply dropout
        outputs = torch.stack(outputs, dim=1)  # (batch_size, seq_len, hidden_dim)
        outputs = self.dropout(outputs)

        # Pass through the fully connected layer
        logits = self.fc(outputs)  # (batch_size, seq_len, tagset_size)
        return logits


In [10]:
from torch.utils.data import DataLoader, TensorDataset

# Convertir les données X et Y en tensores
X_tensor = [prepare_sequence(sentence, word_to_ix) for sentence in X]
Y_tensor = [prepare_tag_sequence(tags, tag_to_ix) for tags in Y]

# Créer un DataLoader
train_data = TensorDataset(torch.nn.utils.rnn.pad_sequence(X_tensor, batch_first=True, padding_value=word_to_ix[PAD_TOKEN]),
                           torch.nn.utils.rnn.pad_sequence(Y_tensor, batch_first=True, padding_value=tag_to_ix[PAD_TOKEN]))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Initialiser les hyperparamètres
embedding_dim = 64
hidden_dim = 64
vocab_size = len(word_to_ix)
tagset_size = len(tag_to_ix)

# Initialiser le modèle
model = POSTagger(embedding_dim, hidden_dim, vocab_size, tagset_size)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tag_to_ix[PAD_TOKEN])  # Ignorer le padding dans la perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.

# ... (rest of the code)

for batch_idx, (sentence, tags) in enumerate(train_loader):
    # Envoyer les données sur le GPU (si disponible)
    sentence, tags = sentence.to(device), tags.to(device)

# Entraînement
epochs = 10
for epoch in range(epochs):
    model.train()  # Passer en mode entraînement
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, (sentence, tags) in enumerate(train_loader):
        # Envoyer les données sur le GPU (si disponible)
        sentence, tags = sentence.to(device), tags.to(device)

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passer les données dans le modèle
        tag_scores = model(sentence)  # Shape: (batch_size, seq_len, tagset_size)

        # Redimensionner tag_scores et tags pour CrossEntropyLoss
        tag_scores = tag_scores.view(-1, tagset_size)  # (batch_size * seq_len, tagset_size)
        tags = tags.view(-1)  # (batch_size * seq_len)

        # Calculer la perte
        loss = criterion(tag_scores, tags)
        loss.backward()
        optimizer.step()

        # Suivi de la perte
        total_loss += loss.item()

        # Calcul de la précision
        predictions = torch.argmax(tag_scores, dim=1)  # Obtenir les indices des classes prédites
        mask = tags != tag_to_ix[PAD_TOKEN]  # Ignorer les positions de padding
        correct_predictions += (predictions[mask] == tags[mask]).sum().item()
        total_predictions += mask.sum().item()

    # Calcul de la perte moyenne et de l'accuracy
    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    # Affichage des résultats pour chaque époque
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 2.0714, Accuracy: 0.5353
Epoch 2/10, Loss: 0.8241, Accuracy: 0.7857
Epoch 3/10, Loss: 0.5652, Accuracy: 0.8449
Epoch 4/10, Loss: 0.4611, Accuracy: 0.8762
Epoch 5/10, Loss: 0.4004, Accuracy: 0.8849
Epoch 6/10, Loss: 0.3695, Accuracy: 0.8879
Epoch 7/10, Loss: 0.3491, Accuracy: 0.8914
Epoch 8/10, Loss: 0.3294, Accuracy: 0.8948
Epoch 9/10, Loss: 0.3095, Accuracy: 0.9007
Epoch 10/10, Loss: 0.2859, Accuracy: 0.9094


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# Initialiser les hyperparamètres
embedding_dim = 100
hidden_dim = 128
vocab_size = len(word_to_ix)
tagset_size = len(tag_to_ix)

# Initialiser le modèle
model = POSTagger(embedding_dim, hidden_dim, vocab_size, tagset_size)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tag_to_ix[PAD_TOKEN])  # Ignorer le padding dans la perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.

# ... (rest of the code)

for batch_idx, (sentence, tags) in enumerate(train_loader):
    # Envoyer les données sur le GPU (si disponible)
    sentence, tags = sentence.to(device), tags.to(device)

# Entraînement
epochs = 10
for epoch in range(epochs):
    model.train()  # Passer en mode entraînement
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, (sentence, tags) in enumerate(train_loader):
        # Envoyer les données sur le GPU (si disponible)
        sentence, tags = sentence.to(device), tags.to(device)

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passer les données dans le modèle
        tag_scores = model(sentence)  # Shape: (batch_size, seq_len, tagset_size)

        # Redimensionner tag_scores et tags pour CrossEntropyLoss
        tag_scores = tag_scores.view(-1, tagset_size)  # (batch_size * seq_len, tagset_size)
        tags = tags.view(-1)  # (batch_size * seq_len)

        # Calculer la perte
        loss = criterion(tag_scores, tags)
        loss.backward()
        optimizer.step()

        # Suivi de la perte
        total_loss += loss.item()

        # Calcul de la précision
        predictions = torch.argmax(tag_scores, dim=1)  # Obtenir les indices des classes prédites
        mask = tags != tag_to_ix[PAD_TOKEN]  # Ignorer les positions de padding
        correct_predictions += (predictions[mask] == tags[mask]).sum().item()
        total_predictions += mask.sum().item()

    # Calcul de la perte moyenne et de l'accuracy
    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    # Affichage des résultats pour chaque époque
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 1.5132, Accuracy: 0.6870
Epoch 2/10, Loss: 0.5578, Accuracy: 0.8507
Epoch 3/10, Loss: 0.4153, Accuracy: 0.8827
Epoch 4/10, Loss: 0.3522, Accuracy: 0.8895
Epoch 5/10, Loss: 0.3223, Accuracy: 0.8951
Epoch 6/10, Loss: 0.2923, Accuracy: 0.9067
Epoch 7/10, Loss: 0.2610, Accuracy: 0.9207
Epoch 8/10, Loss: 0.2288, Accuracy: 0.9316
Epoch 9/10, Loss: 0.1988, Accuracy: 0.9430
Epoch 10/10, Loss: 0.1775, Accuracy: 0.9517


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# Initialiser les hyperparamètres
embedding_dim = 128
hidden_dim = 128
vocab_size = len(word_to_ix)
tagset_size = len(tag_to_ix)

# Initialiser le modèle
model = POSTagger(embedding_dim, hidden_dim, vocab_size, tagset_size)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tag_to_ix[PAD_TOKEN])  # Ignorer le padding dans la perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# If a GPU is available, it will use the GPU. Otherwise, it will use the CPU.

# ... (rest of the code)

for batch_idx, (sentence, tags) in enumerate(train_loader):
    # Envoyer les données sur le GPU (si disponible)
    sentence, tags = sentence.to(device), tags.to(device)

# Entraînement
epochs = 10
for epoch in range(epochs):
    model.train()  # Passer en mode entraînement
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, (sentence, tags) in enumerate(train_loader):
        # Envoyer les données sur le GPU (si disponible)
        sentence, tags = sentence.to(device), tags.to(device)

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passer les données dans le modèle
        tag_scores = model(sentence)  # Shape: (batch_size, seq_len, tagset_size)

        # Redimensionner tag_scores et tags pour CrossEntropyLoss
        tag_scores = tag_scores.view(-1, tagset_size)  # (batch_size * seq_len, tagset_size)
        tags = tags.view(-1)  # (batch_size * seq_len)

        # Calculer la perte
        loss = criterion(tag_scores, tags)
        loss.backward()
        optimizer.step()

        # Suivi de la perte
        total_loss += loss.item()

        # Calcul de la précision
        predictions = torch.argmax(tag_scores, dim=1)  # Obtenir les indices des classes prédites
        mask = tags != tag_to_ix[PAD_TOKEN]  # Ignorer les positions de padding
        correct_predictions += (predictions[mask] == tags[mask]).sum().item()
        total_predictions += mask.sum().item()

    # Calcul de la perte moyenne et de l'accuracy
    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    # Affichage des résultats pour chaque époque
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 1.4374, Accuracy: 0.6660
Epoch 2/10, Loss: 0.5202, Accuracy: 0.8776
Epoch 3/10, Loss: 0.3886, Accuracy: 0.8862
Epoch 4/10, Loss: 0.3408, Accuracy: 0.8905
Epoch 5/10, Loss: 0.3077, Accuracy: 0.8976
Epoch 6/10, Loss: 0.2784, Accuracy: 0.9139
Epoch 7/10, Loss: 0.2446, Accuracy: 0.9278
Epoch 8/10, Loss: 0.2127, Accuracy: 0.9391
Epoch 9/10, Loss: 0.1854, Accuracy: 0.9476
Epoch 10/10, Loss: 0.1613, Accuracy: 0.9553


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
# Prepare the test data (convert to tensors)
X_test_tensor = [torch.tensor([word_to_ix.get(word, word_to_ix[PAD_TOKEN]) for word in sentence], dtype=torch.long) for sentence in X_test]
Y_test_tensor = [torch.tensor([tag_to_ix[tag] for tag in tags], dtype=torch.long) for tags in Y_test]

# Tester avec les données de test
model.eval()  # Mettre le modèle en mode évaluation
with torch.no_grad():
    y_true = []  # Liste des vérités de terrain
    y_pred = []  # Liste des prédictions du modèle
    correct_predictions = 0  # Compteur des prédictions correctes
    total_predictions = 0  # Compteur total des prédictions

    for sentence, tags in zip(X_test_tensor, Y_test_tensor):
        sentence_in = torch.tensor(sentence, dtype=torch.long).unsqueeze(0)  # Ajouter une dimension batch
        tags_in = torch.tensor(tags, dtype=torch.long).unsqueeze(0)  # Ajouter une dimension batch

        # Passer les données à travers le modèle
        tag_scores = model(sentence_in)

        # Obtenir les prédictions
        predicted = torch.argmax(tag_scores, dim=2)  # Supposons que la forme soit correcte

        # Ajouter les résultats aux listes
        y_true.extend(tags_in.cpu().numpy().flatten())  # Convertir en numpy et aplatir
        y_pred.extend(predicted.cpu().numpy().flatten())  # Convertir en numpy et aplatir

        # Calculer les prédictions correctes
        correct_predictions += (predicted == tags_in).sum().item()  # Vérifier si les prédictions sont correctes
        total_predictions += tags_in.size(1)  # Ajouter le nombre total de tokens dans le batch

    # Convertir en numpy pour calculer les métriques
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calcul de l'accuracy globale
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy globale: {accuracy:.4f}")

    # Afficher les métriques classiques
    print(f"Précision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Rappel: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")

# Afficher un rapport complet des métriques pour chaque classe
print("\nClassification Report:")
target_names = list(tag_to_ix.keys())
target_names.remove("<PAD>")  # Retirer le token <PAD> s'il est présent
labels = sorted(list(set(y_true) | set(y_pred)))  # Récupérer les classes uniques

print(classification_report(y_true, y_pred, labels=labels, target_names=target_names))


<ipython-input-17-eba0de651984>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_in = torch.tensor(sentence, dtype=torch.long).unsqueeze(0)  # Ajouter une dimension batch
<ipython-input-17-eba0de651984>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tags_in = torch.tensor(tags, dtype=torch.long).unsqueeze(0)  # Ajouter une dimension batch


Accuracy globale: 0.9575
Précision: 0.9569
Rappel: 0.9575
F1-Score: 0.9522

Classification Report:
               precision    recall  f1-score   support

      SURFACE       0.92      0.75      0.82       448
       MESURE       0.00      0.00      0.00       159
     POSITION       0.95      0.80      0.87       147
    STRUCTURE       0.90      0.05      0.10       176
DEVELOPPEMENT       0.00      0.00      0.00         6
      COULEUR       0.99      0.68      0.81       171
       ORGANE       0.89      0.97      0.92      4157
        FORME       0.81      0.70      0.75       722
  DESCRIPTEUR       0.64      0.81      0.71      1498
          OUT       1.00      1.00      1.00     25907
  DISPOSITION       1.00      0.01      0.01       133

     accuracy                           0.96     33524
    macro avg       0.74      0.52      0.55     33524
 weighted avg       0.96      0.96      0.95     33524



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Exemple de phrase à tester
sentence = "Des feuilles ovales "

# Tokeniser la phrase (en séparant par des espaces, ou en utilisant une autre méthode de tokenisation si nécessaire)
tokens = sentence.lower().split()

# Convertir les tokens en indices (en utilisant le dictionnaire `word_to_ix`)
sentence_in = torch.tensor([word_to_ix.get(word, word_to_ix[PAD_TOKEN]) for word in tokens], dtype=torch.long).unsqueeze(0)  # Ajouter la dimension batch

# Passer la phrase dans le modèle pour obtenir les scores des tags
with torch.no_grad():
    tag_scores = model(sentence_in)

# Obtenir les tags prédits (la classe avec la probabilité maximale)
predicted = torch.argmax(tag_scores, dim=2).squeeze().cpu().numpy()

# Afficher les mots avec leurs tags prédits
for word, tag_idx in zip(tokens, predicted):
    tag = list(tag_to_ix.keys())[list(tag_to_ix.values()).index(tag_idx)]
    print(f"Mot: {word} - Tag Prédit: {tag}")


Mot: des - Tag Prédit: OUT
Mot: feuilles - Tag Prédit: ORGANE
Mot: ovales - Tag Prédit: FORME


In [ ]:
import torch
import torch.nn as nn

class PeepholeLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(PeepholeLSTMCell, self).__init__()
        self.hidden_size = hidden_size

        # GATES
        self.Wxi = nn.Linear(input_size, hidden_size, bias=True)
        self.Whi = nn.Linear(hidden_size, hidden_size, bias=False)
        self.Wci = nn.Parameter(torch.randn(hidden_size))  # Peephole pour it

        self.Wxf = nn.Linear(input_size, hidden_size, bias=True)
        self.Whf = nn.Linear(hidden_size, hidden_size, bias=False)
        self.Wcf = nn.Parameter(torch.randn(hidden_size))  # Peephole pour ft

        self.Wxo = nn.Linear(input_size, hidden_size, bias=True)
        self.Who = nn.Linear(hidden_size, hidden_size, bias=False)
        self.Wco = nn.Parameter(torch.randn(hidden_size))  # Peephole pour ot

        # Candidate cell state
        self.Wxg = nn.Linear(input_size, hidden_size, bias=True)
        self.Whg = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x, h_prev, c_prev):
        # Input gate
        i_t = torch.sigmoid(self.Wxi(x) + self.Whi(h_prev) + self.Wci * c_prev)

        # Forget gate
        f_t = torch.sigmoid(self.Wxf(x) + self.Whf(h_prev) + self.Wcf * c_prev)

        # Candidate cell state
        g_t = torch.tanh(self.Wxg(x) + self.Whg(h_prev))

        # Cell state
        c_t = f_t * c_prev + i_t * g_t

        # Output gate
        o_t = torch.sigmoid(self.Wxo(x) + self.Who(h_prev) + self.Wco * c_t)

        # Hidden state
        h_t = o_t * torch.tanh(c_t)

        return h_t, c_t


In [ ]:
class AdvancedPOSTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, dropout=0.5):
        super(AdvancedPOSTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Couche d'embedding
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # Peephole LSTM Cell (cellule personnalisée)
        self.lstm_cell = PeepholeLSTMCell(input_size=embedding_dim, hidden_size=hidden_dim)

        # Couche fully connected (FC) pour prédire les étiquettes
        self.fc = nn.Linear(hidden_dim, tagset_size)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, sentence):
        # Obtenir les embeddings
        embeds = self.word_embeddings(sentence)  # Shape: (batch_size, seq_len, embedding_dim)

        batch_size, seq_len, _ = embeds.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embeds.device)
        c_t = torch.zeros(batch_size, self.hidden_dim, device=embeds.device)

        outputs = []
        for t in range(seq_len):
            h_t, c_t = self.lstm_cell(embeds[:, t, :], h_t, c_t)  # Utilisation de la cellule LSTM personnalisée
            outputs.append(h_t)

        # Stack outputs and apply dropout
        outputs = torch.stack(outputs, dim=1)  # (batch_size, seq_len, hidden_dim)
        outputs = self.dropout(outputs)

        # Pass through the fully connected layer
        logits = self.fc(outputs)  # (batch_size, seq_len, tagset_size)
        return logits


In [ ]:
# Initialiser les paramètres du modèle
embedding_dim = 100  # Dimension des embeddings
hidden_dim = 128     # Dimension cachée
vocab_size = len(word_to_ix)  # Taille du vocabulaire
tagset_size = len(tag_to_ix)  # Nombre de classes

# Initialiser le modèle
model = AdvancedPOSTagger(
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    vocab_size=vocab_size,
    tagset_size=tagset_size,
    dropout=0.5
).to(device)

# Afficher la structure du modèle
print(model)


AdvancedPOSTagger(
  (word_embeddings): Embedding(13497, 100)
  (lstm_cell): PeepholeLSTMCell(
    (Wxi): Linear(in_features=100, out_features=128, bias=True)
    (Whi): Linear(in_features=128, out_features=128, bias=False)
    (Wxf): Linear(in_features=100, out_features=128, bias=True)
    (Whf): Linear(in_features=128, out_features=128, bias=False)
    (Wxo): Linear(in_features=100, out_features=128, bias=True)
    (Who): Linear(in_features=128, out_features=128, bias=False)
    (Wxg): Linear(in_features=100, out_features=128, bias=True)
    (Whg): Linear(in_features=128, out_features=128, bias=False)
  )
  (fc): Linear(in_features=128, out_features=12, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Initialiser les hyperparamètres
embedding_dim = 100
hidden_dim = 128
vocab_size = len(word_to_ix)
tagset_size = len(tag_to_ix)

# Initialiser le modèle
model = AdvancedPOSTagger(embedding_dim, hidden_dim, vocab_size, tagset_size).to(device)  # Peephole LSTM

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss(ignore_index=tag_to_ix[PAD_TOKEN])  # Ignorer le padding dans la perte
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement
epochs = 10
for epoch in range(epochs):
    model.train()  # Passer en mode entraînement
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch_idx, (sentence, tags) in enumerate(train_loader):
        # Envoyer les données sur le GPU (si disponible)
        sentence, tags = sentence.to(device), tags.to(device)

        # Réinitialiser les gradients
        optimizer.zero_grad()

        # Passer les données dans le modèle
        tag_scores = model(sentence)  # Shape: (batch_size, seq_len, tagset_size)

        # Redimensionner tag_scores et tags pour CrossEntropyLoss
        tag_scores = tag_scores.view(-1, tagset_size)  # (batch_size * seq_len, tagset_size)
        tags = tags.view(-1)  # (batch_size * seq_len)

        # Calculer la perte
        loss = criterion(tag_scores, tags)
        loss.backward()
        optimizer.step()

        # Suivi de la perte
        total_loss += loss.item()

        # Calcul de la précision
        predictions = torch.argmax(tag_scores, dim=1)  # Obtenir les indices des classes prédites
        mask = tags != tag_to_ix[PAD_TOKEN]  # Ignorer les positions de padding
        correct_predictions += (predictions[mask] == tags[mask]).sum().item()
        total_predictions += mask.sum().item()

    # Calcul de la perte moyenne et de l'accuracy
    average_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    # Affichage des résultats pour chaque époque
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 1.6601, Accuracy: 0.6344
Epoch 2/10, Loss: 0.5855, Accuracy: 0.8359
Epoch 3/10, Loss: 0.4050, Accuracy: 0.8820
Epoch 4/10, Loss: 0.3600, Accuracy: 0.8878
Epoch 5/10, Loss: 0.3371, Accuracy: 0.8940
Epoch 6/10, Loss: 0.3228, Accuracy: 0.8980
Epoch 7/10, Loss: 0.2849, Accuracy: 0.9105
Epoch 8/10, Loss: 0.2594, Accuracy: 0.9220
Epoch 9/10, Loss: 0.2330, Accuracy: 0.9319
Epoch 10/10, Loss: 0.2086, Accuracy: 0.9403


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

# Préparer les données de test (convertir en tenseurs)
X_test_tensor = [torch.tensor([word_to_ix.get(word, word_to_ix[PAD_TOKEN]) for word in sentence], dtype=torch.long) for sentence in X_test]
Y_test_tensor = [torch.tensor([tag_to_ix[tag] for tag in tags], dtype=torch.long) for tags in Y_test]

# Mettre le modèle en mode évaluation
model.eval()

with torch.no_grad():
    y_true = []  # Liste des vérités terrain
    y_pred = []  # Liste des prédictions du modèle
    correct_predictions = 0  # Compteur des prédictions correctes
    total_predictions = 0  # Compteur total des prédictions

    for sentence, tags in zip(X_test_tensor, Y_test_tensor):
        # Convertir la phrase et les étiquettes en batch
        sentence_in = sentence.unsqueeze(0).to(device)  # Ajouter une dimension batch et déplacer sur GPU si disponible
        tags_in = tags.unsqueeze(0).to(device)  # Ajouter une dimension batch et déplacer sur GPU si disponible

        # Passer la phrase dans le modèle
        tag_scores = model(sentence_in)

        # Obtenir les prédictions
        predicted = torch.argmax(tag_scores, dim=2).squeeze(0).cpu().numpy()  # Supprimer la dimension batch pour obtenir les prédictions

        # Ajouter les prédictions et les vérités terrain aux listes
        y_true.extend(tags.cpu().numpy().flatten())  # Convertir en numpy et aplatir
        y_pred.extend(predicted.flatten())  # Aplatir les prédictions

        # Calculer les prédictions correctes
        correct_predictions += (predicted == tags.cpu().numpy()).sum()  # Comparer les prédictions avec les vérités terrain
        total_predictions += tags.size(0)  # Ajouter le nombre de tokens dans la phrase

    # Convertir les listes en numpy
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calcul de l'accuracy globale
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy globale: {accuracy:.4f}")

    # Afficher les métriques classiques
    print(f"Précision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Rappel: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")

# Afficher un rapport complet des métriques pour chaque classe
print("\nClassification Report:")
target_names = list(tag_to_ix.keys())
target_names.remove(PAD_TOKEN)  # Retirer le token de padding
labels = sorted(list(set(y_true) | set(y_pred)))  # Récupérer les classes uniques

print(classification_report(y_true, y_pred, labels=labels, target_names=target_names))


Accuracy globale: 0.9492
Précision: 0.9511
Rappel: 0.9492
F1-Score: 0.9422

Classification Report:
               precision    recall  f1-score   support

      SURFACE       0.92      0.71      0.80       448
       MESURE       0.00      0.00      0.00       159
     POSITION       1.00      0.52      0.68       147
    STRUCTURE       1.00      0.01      0.01       176
DEVELOPPEMENT       0.00      0.00      0.00         6
      COULEUR       1.00      0.26      0.42       171
       ORGANE       0.85      0.97      0.91      4157
        FORME       0.77      0.49      0.60       722
  DESCRIPTEUR       0.59      0.80      0.68      1498
          OUT       1.00      1.00      1.00     25907
  DISPOSITION       1.00      0.01      0.01       133

     accuracy                           0.95     33524
    macro avg       0.74      0.43      0.46     33524
 weighted avg       0.95      0.95      0.94     33524



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m